In [ ]:

import numpy as np
import pandas as pd
import random
from sklearn import metrics

# Plots
from matplotlib import pyplot
import matplotlib.pyplot as plt
import seaborn as sns

# Training Models
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn import linear_model, tree, model_selection,ensemble
from sklearn.model_selection import learning_curve
from sklearn.neighbors import BallTree, DistanceMetric



### Subway Data

In [ ]:
subway = pd.read_csv("NYC_Transit_Subway_Entrance_And_Exit_Data.csv")
subway.head(5)


,Division,Line,Station Name,Station Latitude,Station Longitude,Route1,Route2,Route3,Route4,Route5,...,ADA,ADA Notes,Free Crossover,North South Street,East West Street,Corner,Entrance Latitude,Entrance Longitude,Station Location,Entrance Location
0,BMT,4 Avenue,25th St,40.660397,-73.998091,R,NaN,NaN,NaN,NaN,...,False,NaN,False,4th Ave,25th St,SW,40.660489,-73.998220,"(40.660397, -73.998091)","(40.660489, -73.99822)"
1,BMT,4 Avenue,25th St,40.660397,-73.998091,R,NaN,NaN,NaN,NaN,...,False,NaN,False,4th Ave,25th St,SE,40.660323,-73.997952,"(40.660397, -73.998091)","(40.660323, -73.997952)"
2,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,...,False,NaN,True,4th Ave,36th St,NW,40.654676,-74.004306,"(40.655144, -74.003549)","(40.654676, -74.004306)"
3,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,...,False,NaN,True,4th Ave,36th St,NE,40.654365,-74.004113,"(40.655144, -74.003549)","(40.654365, -74.004113)"
4,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,...,False,NaN,True,4th Ave,36th St,NW,40.654490,-74.004499,"(40.655144, -74.003549)","(40.65449, -74.004499)"


In [ ]:
subway.columns

Index(['Division', 'Line', 'Station Name', 'Station Latitude',
       'Station Longitude', 'Route1', 'Route2', 'Route3', 'Route4', 'Route5',
       'Route6', 'Route7', 'Route8', 'Route9', 'Route10', 'Route11',
       'Entrance Type', 'Entry', 'Exit Only', 'Vending', 'Staffing',
       'Staff Hours', 'ADA', 'ADA Notes', 'Free Crossover',
       'North South Street', 'East West Street', 'Corner', 'Entrance Latitude',
       'Entrance Longitude', 'Station Location', 'Entrance Location'],
      dtype='object')

In [ ]:
subway =subway [['Line',"Station Name", "Station Longitude", "Station Latitude"]]

In [ ]:
subway.drop_duplicates(subset=['Station Name'])

,Line,Station Name,Station Longitude,Station Latitude
0,4 Avenue,25th St,-73.998091,40.660397
2,4 Avenue,36th St,-74.003549,40.655144
5,4 Avenue,45th St,-74.010006,40.648939
9,4 Avenue,53rd St,-74.014034,40.645069
14,4 Avenue,59th St,-74.017881,40.641362
...,...,...,...,...
1837,White Plains Road,Freeman St,-73.891865,40.829993
1843,White Plains Road,Intervale Av,-73.896736,40.822181
1846,White Plains Road,Jackson Av,-73.907807,40.816490
1860,White Plains Road,Simpson St,-73.893064,40.824073


### Listings Data

In [ ]:
listings = pd.read_csv('cleaned_listings.csv')
listings

,id,name,description,neighborhood_overview,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,amenities_str
0,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",Just the right mix of urban center and local n...,2008-12-07,a few days or more,0.0,64.0,f,1.0,...,4.45,4.58,4.49,4.78,4.80,4.71,4.64,f,1,"""Hot water"", ""Iron"", ""Children\u2019s books an..."
1,5121,BlissArtsSpace!,The space HELLO EVERYONE AND THANKS FOR VISI...,No Data,2009-02-03,within an hour,100.0,100.0,f,1.0,...,4.52,4.22,4.09,4.91,4.91,4.47,4.52,f,2,"""Heating"", ""Air conditioning"", ""Long term stay..."
2,5136,"Spacious Brooklyn Duplex, Patio + Garden",We welcome you to stay in our lovely 2 br dupl...,No Data,2009-02-03,No Data,NaN,25.0,f,1.0,...,5.00,5.00,5.00,5.00,5.00,4.50,5.00,f,1,"""Cable TV"", ""TV with standard cable"", ""Outdoor..."
3,5178,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,"Theater district, many restaurants around here.",2009-03-03,within a few hours,100.0,100.0,f,1.0,...,4.22,4.21,3.73,4.66,4.42,4.87,4.36,f,1,"""Hot water"", ""Body soap"", ""Iron"", ""Lock on bed..."
4,5203,Cozy Clean Guest Room - Family Apt,"Our best guests are seeking a safe, clean, spa...",Our neighborhood is full of restaurants and ca...,2009-02-05,No Data,NaN,NaN,f,1.0,...,4.91,4.83,4.82,4.97,4.95,4.94,4.92,f,1,"""Fire extinguisher"", ""Hot water"", ""Carbon mono..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33145,54146235,Micro Hostel in The Heart of Chelsea New York,Everything you want to explore is right outsid...,No Data,2017-10-20,within a day,100.0,100.0,f,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,t,4,"""Wifi"", ""Building staff"", ""Long term stays all..."
33146,54146364,Hermoso apartamento en el centro de williamsburg,Disfruta de la sencillez de este alojamiento t...,No Data,2018-12-12,within a few hours,100.0,72.0,f,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,f,2,"""Hot water"", ""Body soap"", ""Iron"", ""Window AC u..."
33147,54148518,Time Square New York -Share Room,New York!!! The trendiest city in the world. ...,The center of Manhattan! There are many uniqu...,2021-12-16,within a few hours,100.0,100.0,f,1.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,t,2,"""Hot water"", ""Ceiling fan"", ""Shower gel"", ""Ref..."
33148,54150715,Private PATIO in PRIVATE Room | 5mins to Manha...,No Data,No Data,2020-03-05,No Data,NaN,NaN,f,12.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,f,1,"""Fire extinguisher"", ""Hot water"", ""Carbon mono..."


In [ ]:
# Build k-d tree with haversine distance metric, which expects
# (lat, lon) in radians and returns distances in radians
dist = DistanceMetric.get_metric('haversine')
tree = BallTree(np.radians(subway[['Station Latitude', 'Station Longitude']]), metric=dist)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/neighbors/_distance_metric.py:10: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be removed in 1.3
  warnings.warn(


In [ ]:
coords = np.radians(listings[['latitude', 'longitude']])
dists, ilocs = tree.query(coords)
# dists is in rad; convert to km
listings['dist_to_subway'] = dists.flatten() * 6367

In [ ]:
listings

,id,name,description,neighborhood_overview,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,amenities_str,dist_to_subway
0,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",Just the right mix of urban center and local n...,2008-12-07,a few days or more,0.0,64.0,f,1.0,...,4.58,4.49,4.78,4.80,4.71,4.64,f,1,"""Hot water"", ""Iron"", ""Children\u2019s books an...",0.401337
1,5121,BlissArtsSpace!,The space HELLO EVERYONE AND THANKS FOR VISI...,No Data,2009-02-03,within an hour,100.0,100.0,f,1.0,...,4.22,4.09,4.91,4.91,4.47,4.52,f,2,"""Heating"", ""Air conditioning"", ""Long term stay...",0.464577
2,5136,"Spacious Brooklyn Duplex, Patio + Garden",We welcome you to stay in our lovely 2 br dupl...,No Data,2009-02-03,No Data,NaN,25.0,f,1.0,...,5.00,5.00,5.00,5.00,4.50,5.00,f,1,"""Cable TV"", ""TV with standard cable"", ""Outdoor...",0.337801
3,5178,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,"Theater district, many restaurants around here.",2009-03-03,within a few hours,100.0,100.0,f,1.0,...,4.21,3.73,4.66,4.42,4.87,4.36,f,1,"""Hot water"", ""Body soap"", ""Iron"", ""Lock on bed...",0.211683
4,5203,Cozy Clean Guest Room - Family Apt,"Our best guests are seeking a safe, clean, spa...",Our neighborhood is full of restaurants and ca...,2009-02-05,No Data,NaN,NaN,f,1.0,...,4.83,4.82,4.97,4.95,4.94,4.92,f,1,"""Fire extinguisher"", ""Hot water"", ""Carbon mono...",0.058776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33145,54146235,Micro Hostel in The Heart of Chelsea New York,Everything you want to explore is right outsid...,No Data,2017-10-20,within a day,100.0,100.0,f,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,t,4,"""Wifi"", ""Building staff"", ""Long term stays all...",0.218969
33146,54146364,Hermoso apartamento en el centro de williamsburg,Disfruta de la sencillez de este alojamiento t...,No Data,2018-12-12,within a few hours,100.0,72.0,f,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,f,2,"""Hot water"", ""Body soap"", ""Iron"", ""Window AC u...",0.443880
33147,54148518,Time Square New York -Share Room,New York!!! The trendiest city in the world. ...,The center of Manhattan! There are many uniqu...,2021-12-16,within a few hours,100.0,100.0,f,1.0,...,0.00,0.00,0.00,0.00,0.00,0.00,t,2,"""Hot water"", ""Ceiling fan"", ""Shower gel"", ""Ref...",0.230963
33148,54150715,Private PATIO in PRIVATE Room | 5mins to Manha...,No Data,No Data,2020-03-05,No Data,NaN,NaN,f,12.0,...,0.00,0.00,0.00,0.00,0.00,0.00,f,1,"""Fire extinguisher"", ""Hot water"", ""Carbon mono...",0.436309


In [ ]:
listings.to_csv('dist_cleaned_listings.csv', index = False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2a92d0af-cecf-4cde-96f5-c3db3a7f88f1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>